# L4b: Lattice Models for Equity Pricing
Today, we will introduce lattice models of equity share prices. We’ll estimate the lattice parameters using data-driven and model-driven approaches. Finally, we explore whether a lattice model shows any of the stylized facts we find in market observations. 

> __Learning Objectives__
>
> By the end of this lecture, you will be able to define and demonstrate mastery of the following key concepts:
> * A __binomial lattice model__ assumes that tomorrow exists in one of two possible states: up or down. From today, we move to the up-state with probability p and the down-state with probability (1 - p).
> * __Lattice model parameters__: There are two possible approaches for the lattice parameters. A real-world probability approach assumes we estimate parameters from data. A risk-neutral probability approach assumes a particular model for the lattice parameters.
> * __Stylized facts__: Stylized facts refer to empirical statistical properties of asset returns across various financial markets, such as heavy tails, volatility clustering, and the lack of autocorrelation. These serve as essential guidelines for developing and validating financial models.
> 
> While lattice models are not the most sophisticated models available, they are easy to understand and implement, and are suprisingly effective in capturing the essential features of equity share prices.

So, let's get started!

___

## Examples
Today, we will be using the following example(s) to illustrate key concepts:

* > [▶ Binomial Lattice Simulation of Equity Share Price using Real-World Probability](CHEME-5660-L4b-LatticeModelSharePrice-RWPM-Example-Fall-2025.ipynb). In this example, we will develop a binomial lattice model of equity share prices using a real-world probability approach. We will simulate the share price over a specified time horizon, and then analyze the results to see if the model captures any of the stylized facts we observe in real market data.
___

<div>
    <center>
        <img src="figs/Fig-Lattice-Schematic.svg" width="800"/>
    </center>
</div>

## Bionomial Lattice Model
A binomial lattice model is a discrete-time model used to represent the evolution of an asset's price over time. 
> __Binomial Lattice Model__: A binomial lattice model assumes that at each time step, the asset can either move up by a factor $u$ with a probability $p$ or down by a factor $d$ with probability $(1-p)$, creating a tree-like structure of possible future prices. At time $t\geq{0}$ there will be $t+1$ possible prices. 

> __Key assumptions__: The probaility $p$, and the up and down factors are independent of the time step $t$, i.e., these parameters are constant across the lattice. The up, down and probability parameters are specific to each firm.

Suppose we have a stock with an initial price $S_0$ at time $t=0$ (today). Assuming a binomial lattice model, at $t = 1$ (tomorrow), the stock can occupy one of two possible states: the up state with a price $S^{(+)}_{1} = S_{0}\;u$ with probability $p$ or the down state with price $S^{(-)}_{1} = S_{0}\;d$ with probability $(1-p)$. 

Future prices at time $t = 1$ can be expressed as:
$$
S_{1} = \begin{cases}
S^{(+)}_{1} = S_{0}\;u & \text{with probability } p \\
S^{(-)}_{1} = S_{0}\;d & \text{with probability } (1-p)
\end{cases}
$$
The process continues for $n$ time steps, creating a binomial tree of possible prices.

### Price Distribution
The binomial lattice model gets its name because the prices at level $t$ can be represented as a binomial distribution. Let's see where this comes from. 

> __Key idea__: At each node of the lattice, we are performing a Bernoulli trial: we either move up with probability $p$ or down with probability $(1-p)$. Thus, we have a sequence of $t$ independent Bernoulli trials, where each trial has two possible outcomes: up or down. This sequence of trials can be represented as a binomial distribution.

Consider a path from the root ($t=0$) to any node at level $t$. This path consists of exactly $t$ moves, where each move is either up (U) or down (D). Let $k$ be the number of up moves and $(t-k)$ be the number of down moves. The price at any node with $k$ up moves and $(t-k)$ down moves is:
$$
S_{t,k} = S_0 \cdot u^k \cdot d^{t-k}
$$
The probability of reaching this specific node is given by the binomial probability:
$$
P(k \text{ up moves in } t \text{ steps}) = \binom{t}{k}\;p^k (1-p)^{t-k}
$$

where $\binom{t}{k} = \frac{t!}{k!(t-k)!}$ is the binomial coefficient representing the number of ways to choose $k$ up moves from $t$ total moves.
At level $t$ (e.g., $t$ days in the future), there are $(t+1)$ possible prices:
$$
\begin{align*}
S_{t,0} &= S_0 d^t &&\text{(all down moves)} \\
S_{t,1} &= S_0 u^1 d^{t-1} &&\text{(one up, } (t-1) \text{ down)} \\
&\vdots \\
S_{t,t} &= S_0 u^t &&\text{(all up moves)}
\end{align*}
$$
The complete price distribution at level $t$ follows:
$$
\boxed{
S_t = S_0 u^k d^{t-k} \quad \text{with probability} \quad \binom{t}{k}\;p^k (1-p)^{t-k}, \quad k = 0,1,\ldots,t
}
$$

This is why it's called a **binomial** lattice model - the price distribution at each level follows a binomial distribution with parameters $t$ (number of trials) and $p$ (probability of success/up move).

So where do we get the up and down factors $u$ and $d$, and the probability $p$? There are two approaches: a data-driven approach and a model-driven approach. Let's start with the data-driven approach.

### Real World Parameter Estimation
For us to simulate future share price (and return/growth rate) distributions, we need to estimate the up and down factors $u$ and $d$, and the probability $p$. We can estimate these parameters $(u,d,p)$ from historical data.

Let's estimate the up $u$, down $d$ factors and the probability $p$ from historical data.

__Initialize__: Given the growth rate sequence $\{\mu_{2},\mu_{3},\dots,\mu_{T}\}$ for firm $(i)$ (we neglect the superscript $i$ for simplicity) and a time step $\Delta{t} > 0$ (units: years). Initialize the up factors collection $U = \emptyset$ and down factors collection $D = \emptyset$.

1. For $t = 2,3,\dots,T$ __do__:
    - If $\mu_{t} > 0$, then update $U \gets U \cup \{e^{\mu_{t}\Delta{t}}\}$ where $U$ is a collection of up factors.
    - If $\mu_{t} < 0$, then update $D \gets D \cup \{e^{\mu_{t}\Delta{t}}\}$ where $D$ is a collection of down factors.
    - If $\mu_{t} = 0$, skip (no price change).
2. Compute the up factor $u$ as the mean of the up factors collection $U$: $u = \frac{1}{|U|} \sum_{v \in U} v$.
3. Compute the down factor $d$ as the mean of the down factors collection $D$: $d = \frac{1}{|D|} \sum_{v \in D} v$.
4. Compute the probability $p$ as the fraction of up movements: $p = \frac{|U|}{|U| + |D|}$.

> __Note on recombination__: This data-driven approach produces a recombining lattice since $ud = du$ (multiplication is commutative). Later, we'll impose the constraint $ud = 1$, however, this condition is not necessary for recombination, though it may be imposed in some models for risk-neutral pricing or to ensure symmetric price movements.

> __Example__
>
> [▶ Binomial Lattice Simulation of Equity Share Price using Real-World Probability](CHEME-5660-L4b-LatticeModelSharePrice-RWPM-Example-Fall-2025.ipynb). In this example, we will develop a binomial lattice model of equity share prices using a real-world probability approach. We will simulate the share price over a specified time horizon, and then analyze the results to see if the model captures any of the stylized facts we observe in real market data.

___


## Stylized Facts for Binomial Lattice Models
Since we have an anlytical expression for the price distribution at time $t$, we can also derive the growth rate distribution. We know the growth rate over a time period $\Delta{t}$ is given by:
$$
\begin{align*}
\mu_{t} &= \left(\frac{1}{\Delta{t}}\right)\;\ln\left(\frac{S_{t}}{S_{0}}\right)\quad\Longrightarrow\text{substitute}\;S_{t} = S_{0}u^k d^{t-k} \\
& = \left(\frac{1}{\Delta{t}}\right)\;\ln\left(\frac{S_{0}u^k d^{t-k}}{S_{0}}\right)\\
& = \left(\frac{1}{\Delta{t}}\right)\;\ln\left(u^k d^{t-k}\right)\\
& = \left(\frac{1}{\Delta{t}}\right)\;\Bigl[k\ln(u) + (t-k)\ln(d)\Bigr]\\
& = \left(\frac{1}{\Delta{t}}\right)\;\Bigl[k\ln\left(\frac{u}{d}\right)+t\ln(d)\Bigr]\\
\end{align*}
$$
The growth rate at time $t$ is a linear function of the number of up moves $k$ and the total number of moves $t$. However, the number of up moves is a random variable $k \sim \texttt{Binomial}(t,p)$, thus, the growth rate distribution is the discrete random variable:
$$
\boxed{
\mu_{t} = \left(\frac{1}{\Delta{t}}\right)\;\Bigl[k\ln\left(\frac{u}{d}\right)+t\ln(d)\Bigr] \quad\text{where}\;k\sim \texttt{Binomial}(t,p)
}
$$

#### Expected Value and Variance of Growth Rate Distribution
Let's look at the expected value of the growth rate distribution. Let's take the expectation of the growth rate distribution:
$$
\begin{align*}
\mathbb{E}[\mu_t]
& =\frac{1}{\Delta t}\left[\mathbb{E}[k]\;\ln\!\Big(\frac{u}{d}\Big)+t\ln d\right]\quad\;k=\sum_{i=1}^t X_i;\text{with}\;X_i\stackrel{\text{i.i.d.}}{\sim}\mathrm{Bernoulli}(p)\rightarrow\mathbb{E}[k]=\sum_{i=1}^t \mathbb{E}[X_i]=tp\\
& =\frac{1}{\Delta t}\left[tp\ln\!\Big(\frac{u}{d}\Big)+t\ln d\right]\\
& =\frac{t}{\Delta t}\left[p\ln\!\Big(\frac{u}{d}\Big)+\ln d\right]\quad\;\Longrightarrow\ln d+p\ln(u/d)=p\ln u+(1-p)\ln d\\
&= \frac{t}{\Delta t}\Big(p\ln u+(1-p)\ln d\Big)\quad\blacksquare
\end{align*}
$$

In a similar manner, we can compute the variance of the growth rate distribution/ Starting from the identity $\operatorname{Var}(aX+b)=a^2\operatorname{Var}(X)$, we have:
$$
\begin{align*}
\operatorname{Var}(\mu_t)
& =\frac{1}{(\Delta t)^2}\,\big(\ln(u/d)\big)^2\,\operatorname{Var}(k)\quad\operatorname{Var}(k)=t\,p(1-p)\\
&=\frac{t\,p(1-p)}{(\Delta t)^2}\,\big(\ln(u/d)\big)^2\\
&=\frac{t\,p(1-p)}{(\Delta t)^2}\,\big(\ln u-\ln d\big)^2\quad\blacksquare
\end{align*}
$$
The variance scales linearly in $t$, vanishes as $p\to 0$ or $1$, and disappears when $u=d$.

We know that equity share price returns (growth rates) exhibit certain statistical properties, known as stylized facts. These properties are observed across various financial markets and are essential for developing and validating financial models.

* __Heavy (also called fat) tailed distribution__: Stock returns often exhibit a distribution with fatter tails than would be expected under a normal distribution. This means that extreme price movements are more likely than would be predicted by a normal distribution.
* __Absence of Autocorrelation__: Autocorrelation refers to the tendency of stock returns to correlate with past returns over time. Autocorrelation suggests predictability in returns, which traders could exploit. On the other hand, a random walk is uncorrelated with itself. We expect a random walk with occasional jumps.
* __Volatility clustering__: Stock returns tend to be more volatile during specific periods and less volatile during others. This phenomenon is known as volatility clustering, suggesting that large price movements are more likely to be followed by other large moves, and other small moves follow small moves.

Does the binomial lattice model capture these stylized facts? Let's explore this question.

### Heavy Tails
The binomial lattice return (growth rate) model does __not__ capture heavy tails.

> __Why no heavy tails?__ The growth rate distribution $\mu_{t}$ is a linear transformation of a binomially distributed variable $k$. As $t \to \infty$, the distribution of $\mu_{t}$ converges to a normal distribution by the Central Limit Theorem, which has lighter tails than the empirical return distributions observed in financial markets.

### Vanishing Autocorrelation
To explore this question, let's think about a share price time series $\left\{S_{0},S_{2},\ldots,S_{t}\right\}$ generated by a binomial lattice model. By induction, we can show:
$$
\begin{align*}
S_{t} = S_{0} \cdot \exp\left(\Delta{t}\;\sum_{i=1}^{t}\mu_{i}\right) \\
\end{align*}
$$
where the sum $\sum_{i=1}^{t}\mu_{i}$ is a sum of independent and identically distributed (i.i.d.) random variables.

### Volatility Clustering
Fill me in later.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.

___